# The Planet API 101: An Introduction for Complete Beginners

![Alt text](https://news.stanford.edu/wp-content/uploads/2020/02/CC154.jpg "Lake Lagunita, circa 1903-1906")

"The Planet API 101: An Introduction for Complete Beginners" is a Python notebook designed to guide novices through using satellite imagery to observe environmental changes.

Specifically, it focuses on analyzing the Winter 2022-23 Atmospheric River Events in Northern California and their impact on the flooding of Lake Lagunita on the Stanford Campus.

Through a step-by-step approach, beginners learn to search, filter, order, and visualize satellite data, offering insights into the inundation event and demonstrating the power of geospatial analysis with real-world applications.

This guide is an adaptation of a [workshop given by Planet.com's Developers for GISDay@Stanford 2023](https://hello.planet.com/data/s/sdtjXQAPQpmBBbc).

This tutorial is an introduction to [Planet](https://www.planet.com)'s Data and Orders API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `planet` module. It shows you how to create bulk orders, use Planet Labs' tools, and deliver to the cloud.

## Requirements

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Python modules used in this tutorial are:
* [IPython](https://ipython.org/) and [Jupyter](https://jupyter.org/)
* [planet](https://github.com/planetlabs/planet-client-python)
* [geojsonio](https://pypi.python.org/pypi/geojsonio)
* [rasterio](https://rasterio.readthedocs.io/en/latest/index.html)
* [asyncio](https://docs.python.org/3/library/asyncio.html)

You should also have an account on the Planet Platform and retrieve your API key from your [account page](https://www.planet.com/account/).

## Useful links
* [Planet Client V2 Documentation](https://github.com/planetlabs/planet-client-python)
* [Planet Data API reference](https://developers.planet.com/docs/apis/data/)

This tutorial will cover the basic operations possible with the Python client, particularly those that interact with the Data API and Orders API

## Set up

In order to interact with the Planet API using the client, we need to import the necessary packages & define helper functions.

In [ ]:
#general packages
import os
import json
import glob
import asyncio
import requests
import nest_asyncio
import matplotlib.pyplot as plt
from requests.auth import HTTPBasicAuth
from datetime import datetime, timedelta
from google.colab import userdata

#geospatial packages
import rasterio
import geopandas as gpd
from shapely.geometry import shape
from shapely.ops import unary_union
import folium


#planet SDK
from planet import Auth, reporting, Session, OrdersClient, order_request, data_filter


# We will also create a small helper function to print out JSON with proper indentation.
def indent(data):
    print(json.dumps(data, indent=2))

# Installing packages

You probably encountered some errors after running that cell. Here, we will install the missing packages. If any other packages were missing from your Python Environment, they should be listed in the error messages, and you can simply add them to the end of the `!pip` command line to add them to the install sequence.

Run the following code cell, then return to the previous cell and try it again.

Continue to install missing packages, using the `!pip install` command, until you no longer recieve error messages.

In [ ]:
# You can install packages that aren't currently installed in your Python Notebook using !pip install <package name>
# In this case, we will install the Planet Package:
!pip install rasterio planet folium

## Authentication

Here, you will paste your API Key when prompted. It will be used to authenticate when ordering data.

Be sure to go to **Edit>Clear all outputs** to clear the console output that results, before sharing this notebook, or uploading it to a public repository, such as GitHub.

Additionally, regularly resetting your API Key on Planet.com can help keep your account and access secure.

You can also authenticate via the CLI using [`auth init`](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/cli/cli-reference/?h=auth#auth:~:text=message%20and%20exit.-,auth,-%C2%B6), this will store your API key as an environment variable.

In [ ]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if 'PL_API_KEY' in os.environ:
    API_KEY = os.environ['PL_API_KEY']
else:
    API_KEY = input("PASTE_API_KEY_HERE AND HIT RETURN:   ")
    os.environ['PL_API_KEY'] = API_KEY

client = Auth.from_key(API_KEY)


Using Google Colab Secret Manager

In [ ]:
from google.colab import userdata
#Store your API key in the Colabs Secret Manager, as PL_API_KEY and enable notebook access to the secret
#Be sure to toggle on Notebook Access in the Secret Manager
# Get the API key from the secret manager
API_KEY = userdata.get('PL_API_KEY')

# Set the API key as an environment variable
os.environ['PL_API_KEY'] = API_KEY

# Create a client for the Planet API
client = Auth.from_key(API_KEY)

# 2022-23 Precipitation Events

![Rainbow over Stanford Campus](https://punchmagazine.com/wp-content/uploads/0T1A7331-lagunita-with-rainbow-from-casbs-big-02-27-2023-Canon-EOS-7D-Mark-II-Robert-Siegel-1-1200x801.jpg "Rainbow over Stanford Campus")

[During the winter of 2022-23, Northern California experienced a series of intense "Atmospheric River" events](https://en.wikipedia.org/wiki/2022%E2%80%932023_California_floods), which led to significant flooding across the region. These meteorological phenomena, characterized by long, narrow regions in the atmosphere that transport most of the water vapor outside of the tropics, brought heavy rains to areas that were not accustomed to such volumes of water.

[Lake Lagunita](https://en.wikipedia.org/wiki/Lake_Lagunita), located on the Stanford University campus, which is often dry and used for recreational activities, became fully inundated. This unexpected transformation highlighted the power of these atmospheric rivers and their impact on both natural and urban environments, showcasing the importance of preparedness and resilience in the face of increasingly unpredictable weather patterns

## Searching an Area of Interest (AOI)

This code reads a **GeoJSON** file named `"lakelagunita.geojson"`
into a Python object using the `json.loads(f.read())` method, which converts the file's JSON content into a Python dictionary. It then accesses the `'features'` list from this dictionary, selects the first feature (`[0]` indicating the first item in the list), and retrieves the `'geometry'` information associated with that feature. Essentially, it extracts the geometric data (such as coordinates defining points, lines, or polygons) of the first geographic feature described in the GeoJSON file.

The `geom_all` variable stores the geometry of the first feature from the file, demonstrating how to access nested data in JSON format. The `geom_large` dictionary defines a new geometry directly in the code, illustrating how to construct a GeoJSON feature programmatically. You could use either of these methods for providing the **Area of Interest (AOI)**



Let's also read in a GeoJSON geometry into a variable so we can use it during testing. *The geometry can only have one polygon to work with the data API*

In [ ]:
with open("lakelagunita.geojson") as f:
    geom_all = json.loads(f.read())['features'][0]['geometry']
geom_large = {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -122.1828114547462,
              37.4264632718164
            ],
            [
              -122.1828114547462,
              37.417673150532494
            ],
            [
              -122.16877378345552,
              37.417673150532494
            ],
            [
              -122.16877378345552,
              37.4264632718164
            ],
            [
              -122.1828114547462,
              37.4264632718164
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
print(geom_all)

## Display the AOI



In [ ]:
m = folium.Map([37.4264632718164,-122.1828114547462], zoom_start=15, tiles="cartodbpositron")

geojson_data = 'lakelagunita.geojson'

folium.GeoJson(geojson_data, name="Lake Lagunita").add_to(m)

folium.LayerControl().add_to(m)

m

## Creating a Filter

This code block sets up filters for querying geospatial data:

- **Item Types**: Defines the type of satellite imagery to search for, in this case, "PSScene".
- **Geometry Filter**: Uses a predefined geometry (`geom_large`) to filter data to only include imagery that intersects with this area.
- **Date Range Filter**: Specifies a date range to filter the imagery, selecting images acquired between December 10, 2022, and September 30, 2023.
- **Combined Filter**: Combines the geometry and date range filters using an "AND" logic, meaning both conditions must be met for an image to be included in the search results.

The cloud cover filter is *commented out*, indicating it's not currently used but can be applied to restrict results to images with less than 80% cloud cover.

The possible filters include `and_filter`, `date_range_filter`, `range_filter` and so on, mirroring the options supported by the Planet API. Additional filters are described [here](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/#filter:~:text=(main())-,Filter,-%C2%B6).

In [ ]:
# Define the filters we'll use to find our data

item_types = ["PSScene"]

#Geometry filter
geom_filter = data_filter.geometry_filter(geom_large)

#Date range filter
date_range_filter = data_filter.date_range_filter(
    "acquired", gt = datetime(month=12, day=10, year=2022),
    lt = datetime(month=9, day=30, year=2023))
#Cloud cover filter
#cloud_cover_filter = data_filter.range_filter('clear_percent', gt = 80)

#Combine all of the filters
combined_filter = data_filter.and_filter([geom_filter, date_range_filter])#, cloud_cover_filter])

## Print the `combined_filter`

Print the filter so you can see what the results look like

In [ ]:
combined_filter

## Set the `item_types`

This code block is setting up an API request object for querying a geospatial data service. It specifies the type of data to search for, "PSScene", using the "item_types" key. Additionally, it includes a "filter" key that incorporates a previously defined combined_filter, which likely combines several criteria (like geographic area, date range, etc.) to narrow down the search results. This request object can then be used with the service's API to fetch data that matches the given criteria.

* "PSScene", Planetscope Scenes  
* "REOrthoTile" for RapidEye OrthoTiles,   
* "REScene" for unorthorectified RapidEye strips,  
* "SkySatScene" for SkySat imagery,  
* "SkySatCollect" for orthorectified SkySat composites  

Additional `item_types` can be found at https://developers.planet.com/docs/apis/data/items-assets/

In [ ]:
item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type],
  "filter": combined_filter
}

In [ ]:
search_request

## POST to the Planet API

This code sends a POST request to the Planet API to search for images matching specific criteria defined in search_request. It uses basic authentication with an API key. The response, assumed to contain image data, is parsed from JSON to extract and print the number of image IDs found, showing how many images matched the search filters.

In [ ]:
# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(API_KEY, ''),
    json=search_request)

# extract image IDs only
image_ids = [feature['id'] for feature in search_result.json()['features']]
print(len(image_ids))

## Pagination links

This code accesses the pagination links from the JSON response of a search query made to an API.

Pagination is used to break down large datasets into smaller, manageable chunks or "pages" of data. `_links` would typically contain URLs to navigate through these pages, allowing the client to request subsequent sets of results (like "next" page or "previous" page) without retrieving all data at once. This is efficient for both the server and client, especially when dealing with large amounts of data.

In [ ]:
search_result.json()['_links']

# Using the SDK

Using the REST API directly requires manually crafting HTTP requests, handling authentication, parsing responses, and managing session states, offering more control and flexibility but requiring more code to handle the interaction.

In contrast, using the SDK (Software Development Kit) abstracts and simplifies the process of interacting with the API by providing pre-built functions and methods, handling low-level details like session management and request retries. It allows for more Pythonic code, with asynchronous capabilities and direct integration into Python applications.

This code performs an asynchronous search request using the Planet SDK, retrieving a list of items that match the specified combined_filter and item_types criteria. It uses an asynchronous session to make the request and asynchronously iterates over the search results up to a limit of 500 items, gathering them into a list called `item_list`. This approach allows for non-blocking network requests, making the code efficient for handling I/O-bound tasks like web requests in a concurrent manner.

If the number of items requested is more than 500, the client will automatically fetch more pages of results in order to get the exact number requested.

Then we can save the output to be visualized as a geojson

In [ ]:
async with Session() as sess:
    cl = sess.client('data')
    item_list = [i async for i in cl.search(search_filter=combined_filter, item_types=item_types,limit=500)]

## Print the # of items in your Search Result

In [ ]:
len(item_list)

## Adding our previous Filters

This code block sets up a cloud cover filter to only include images with greater than 80% clarity, combines it with other filters (geometric and date range), and then performs an asynchronous search with the Planet API to retrieve up to 500 items matching these criteria. It uses an asynchronous session for efficient network operations, collecting the search results into a list named item_list.

In [ ]:
#Cloud cover filter
cloud_cover_filter = data_filter.range_filter('clear_percent', gt = 80)

#Combine all of the filters
combined_filter = data_filter.and_filter([geom_filter, date_range_filter, cloud_cover_filter])

async with Session() as sess:
    cl = sess.client('data')
    item_list = [i async for i in cl.search(search_filter=combined_filter, item_types=item_types,limit=500)]

## Print the newly filtered # of items in your Search Result

In [ ]:
len(item_list)

## Print the Search results

Now, we can iterate through our search results.

This code iterates through the list of items, `item_list`, and prints each item's ID and item type. It accesses the 'id' directly from each item dictionary and the 'item_type' from the nested 'properties' dictionary within each item.

In [ ]:
for item in item_list:
    print(item['id'], item['properties']['item_type'])

## Save all of our scene footprints as a GeoJSON file

This code block creates a **GeoJSON** file representing a collection of spatial features (like satellite images or scenes) from item_list. It first checks if an 'output' directory exists, creating it if not. If a file named `'results01.geojson'` already exists in this directory, it deletes the file. Then, it iterates over item_list, constructing GeoJSON feature objects for each item by including their geometry and properties, and appends these to a feature collection. Finally, it writes this collection as a GeoJSON string to 'results02.geojson'.

In [ ]:
scene_geoms = {
  "type": "FeatureCollection",
  "features": []
}

if not os.path.isdir('output'):
    os.mkdir('output')
else:
    if os.path.isfile('output/results01.geojson'):
        os.remove('output/results01.geojson')

with open('output/results01.geojson','w') as f:
    for item in item_list:
        geom_out =     {
          "type": "Feature",
          "properties": item['properties'],
          "geometry": item['geometry']
        }
        scene_geoms['features'].append(geom_out)
    jsonStr = json.dumps(scene_geoms)
    f.write(jsonStr)
    f.close()

# Display the image footprints

In [ ]:
import requests

m = folium.Map([37.4264632718164,-122.1828114547462], zoom_start=10, tiles="cartodbpositron")

geojson_data = './output/results01.geojson'

folium.GeoJson(geojson_data, name="Image Footprints").add_to(m)

folium.LayerControl().add_to(m)

m

# Examine an Item

The code `item_list[0]` accesses the first item in the list named `item_list`, expected to contain data about geospatial features, specifically from the Planet API. The output shown is a Python dictionary representing a geospatial feature, including links to its data (`_links`), permissions available (`_permissions`), a list of `assets`, the `geometry` defining its spatial footprint, a unique identifier (`id`), and various properties such as a`cquisition time`, `cloud cover`, and more, related to the satellite image.


In [ ]:
item_list[0]

## Ordering

Searching using the Data API involves querying the Planet database to find satellite images that match specific criteria (like date, location, and cloud cover). It's about discovering what data is available.

Ordering using the Orders API, on the other hand, is the next step after identifying the desired images. It involves requesting the processing and delivery of specific datasets, possibly with additional operations like format conversion or applying specific filters, to make the data ready for analysis or integration into applications.

Now that we have all of the imagery that we want to order we need to package it in a way that the Orders API can handle. Breaking it up by week.

This code organizes a list of items (each representing a satellite image with an acquisition date) into groups based on their acquisition dates, with each group covering a span of 30 days.

It first sorts the items by date in ascending order.

Then, it iterates through these items, grouping them together if they fall within the same 30-day period.

If an item's date is outside the current 30-day window, it starts a new group.

Finally, it prints the number of these groups.

In [ ]:
grouped_items = []
current_group = []
#reverse the list since it comes in last date first
reversed_items = sorted(item_list, key=lambda item: item['properties']['acquired'])

#Select the earliest item
group_start_date = datetime.strptime(reversed_items[0]['properties']['acquired'], "%Y-%m-%dT%H:%M:%S.%fZ")

for item in reversed_items:
    time_object = item['properties']['acquired']
    time = datetime.strptime(time_object, "%Y-%m-%dT%H:%M:%S.%fZ")

    if time < group_start_date + timedelta(days=30):
        current_group.append(item)

    else:
        grouped_items.append(current_group)
        current_group = [item]
        group_start_date = time
if current_group:
    grouped_items.append(current_group)

print(len(grouped_items))

## Querying a property of our image groups

Lets see what the cloud cover is for our scenes

This code iterates over the first group of satellite images in `grouped_items` and prints the `clear_percent` property for each image. The clear_percent indicates the percentage of the image not obscured by clouds, providing insight into the image's clarity and suitability for analysis or visual inspection.

The output represents the `clear_percent` values of satellite images from the first group in grouped_items. Each number indicates the percentage of the image area that is free from cloud cover, with higher numbers suggesting clearer conditions. The values range from 81% to 100%, indicating varying levels of clarity across the images, with several images having very high clarity (99% to 100%).

In [ ]:
for item in grouped_items[0]:
    print(item['properties']['clear_percent'])

# Sort the images to prioritize the clearer images

This code sorts each group of satellite images within `grouped_items` by their `clear_percent` value in descending order, ensuring each group's most unobscured images are listed first. It then compiles these sorted groups into a new list, `sorted_items`. Finally, it prints the `clear_percent` values of all images in the first sorted group, displaying them from the highest to the lowest percentage of clarity.

In [ ]:
sorted_items = []
for group in grouped_items:
    sorted_group = sorted(group, key=lambda item: item['properties']['clear_percent'], reverse=True)
    sorted_items.append(sorted_group)


for item in sorted_items[0]:
    print(item['properties']['clear_percent'])

## A function to calculate intersection

This code defines a function get_overlap that calculates the area of overlap between two geometries. It uses the shape function from the shapely.geometry module to convert the input geometries into shape objects. Then, it computes the intersection of these two shapes, which represents the overlapping area, and returns this intersection as the result.

In [ ]:
def get_overlap(geometry1, geometry2):
    """Calculate the area of overlap between two geometries."""
    shape1 = shape(geometry1)
    shape2 = shape(geometry2)

    # Compute the intersection of the two geometries.
    intersection = shape1.intersection(shape2)

    return intersection

## Evaluating coverage

Look for the minimum about on scenes to cover the entire AOI

This code iterates through groups of satellite images (`sorted_items`), selecting a minimum set of images per group that collectively cover the target area (`geom_all`). For each image, it checks if its geometry overlaps with the target area. If there's an existing overlap (`intersection`), it calculates the union of the new and existing overlaps, adding the image to the weekly list if the union expands the covered area.

The goal is to compile lists (`minimum_sorted_list`) of the fewest images needed to cover the target area each week, optimizing for spatial coverage and minimizing cloud cover.


In [ ]:
minimum_sorted_list = []


for week_items in sorted_items:
    intersection = False
    weekly_minimum_list = []
    for item in week_items:
        #for each scene itterate through every geometry and check if it overlaps with the scene
        overlap = get_overlap(geom_all, item['geometry'])
        if intersection:
            new_intersection = unary_union([overlap,intersection])

            #If the new interseciton is bigger then the old then add the scene to the order
            if round(new_intersection.area, 8) > round(intersection.area, 8):
                intersection = new_intersection
                weekly_minimum_list.append(item)
        else:
            if overlap.area > 0:
                intersection = overlap
                weekly_minimum_list.append(item)
    print(len(week_items), " to ", len(weekly_minimum_list))

    if len(weekly_minimum_list) > 0:
        minimum_sorted_list.append(weekly_minimum_list)

## Evaluating cloud cover before and after

This code block is comparing and displaying the clear_percent properties of satellite images from two different lists: `sorted_items[0]` and `minimum_sorted_list[0]`.

The first loop prints the `clear_percent` of each image in the first sorted group, showing how clear each image is.

After printing "Now", the second loop does the same for the first group of images that have been determined to minimally cover a target area, likely optimized for both coverage and clarity.

This allows for a comparison of image clarity before and after optimization.

In [ ]:
for item in sorted_items[0]:
    print(item['properties']['clear_percent'])
print("Now")
for item in minimum_sorted_list[0]:
    print(item['properties']['clear_percent'])

## Clarity of the final selections

Now lets print the average clear percent of each scene we are ordering.

This code calculates and prints the average `clear_percent` for each group of satellite images in `minimum_sorted_list`.

It iterates through each group, collecting the `clear_percent` values into a list, then computes the average of these values for the group, indicating the overall clarity of images selected for minimal coverage.

In [ ]:
for group in minimum_sorted_list:
    clear = []
    for item in group:
        clear.append(int(item['properties']['clear_percent']))
    print(sum(clear)/len(clear))

## Order images for mosaicking with clearer images on top

We need to reverse the order of the scenes one more time because when mosaicing the last scene is stacked at the top.

This code sorts each group of satellite images in `minimum_sorted_list` by their `clear_percent` in ascending order, to ensure that when these images are used in a mosaic, the clearest images (last in the sorted list) are placed on top.

After sorting, it prints the `clear_percent` of each image in the first sorted group, demonstrating the order in which they will be layered in the mosaic, with lower clarity images at the bottom and higher clarity images on top.

In [ ]:
order_items = []
for group in minimum_sorted_list:
    sorted_group = sorted(group, key=lambda item: item['properties']['clear_percent'])
    order_items.append(sorted_group)

for item in order_items[0]:
    print(item['properties']['clear_percent'])

# Place a Order
Create the order structure using `planet` functions

## Create an `assemble_order()' function and test it

This code defines an asynchronous function assemble_order that constructs a request for ordering satellite imagery from the Planet API.

It specifies the image IDs to be included in the order, applies a series of processing tools (clipping to a specified area, performing a band math operation, and creating a composite), and builds the order request with these specifications.

The function then awaits this order assembly process with a test order name and a specific image ID, preparing the order request for submission.

In [ ]:
async def assemble_order(name,item_ids):
    products = [
        order_request.product(item_ids, 'analytic_udm2', 'PSScene')
    ]

    clip = order_request.clip_tool(aoi=geom_all)
    bandmath = order_request.band_math_tool(b1='(b2-b4)/(b2+b4)*100+100', pixel_type='8U')
    composite = order_request.composite_tool()



    tools = [clip,bandmath,composite]

    request = order_request.build_request(
        name, products=products, tools=tools)
    return request

request =  await assemble_order("test",['20230207_180504_51_24b6'])

`print(request)` statement will output the assembled order request details created by the assemble_order function.

This will include the name of the order, specified products (image IDs with their types and item type), and the tools applied (clip, band math, and composite) along with their configurations.

The result is a structured representation of the order that is ready to be submitted to the Planet API for processing.

In [ ]:
request

## Create a function to order imagery

This code defines an asynchronous function, do_order, which takes an order request as input, creates an order with the Planet API using an OrdersClient session, waits for the order to complete, and then downloads the ordered data to a directory named after the order.

It handles the order creation, monitoring, and downloading process asynchronously, allowing for efficient management of potentially long-running network operations involved in ordering satellite imagery.

In [ ]:
async def do_order(request):
    async with Session() as sess:
        cl = OrdersClient(sess)
        #with reporting.StateBar(state='creating') as bar:
        order = await cl.create_order(request)
        #bar.update(state='created', order_id=order['id'])

        await cl.wait(order['id'],max_attempts=0)#, callback=bar.update_state)
        os.mkdir(request['name'])

        # if we get here that means the order completed. Yay! Download the files.
        await cl.download_order(order['id'],directory=request['name'])




## Create all orders

This code iterates over groups of satellite images (`order_items`), constructing a unique order name for each group based on a prefix and the acquisition date of the first image in each group.

It then creates an order for each group of images by calling the `assemble_order()` function with the constructed order name and the IDs of the images in the group, appending the future object representing the asynchronous operation to `order_list`.

Finally, it prints the total number of orders prepared.

In [ ]:
order_list = []
folder_list= []
name = "lake_lagunita_cloud_"
for group in order_items:
    ids = []
    order_name = name + group[0]['properties']['acquired'][:10]
    print(order_name)
    folder_list.append(order_name)
    for item in group:
        ids.append(item['id'])
    order_list.append(await assemble_order(order_name,ids))
print(len(order_list))

## Submitting and monitoring the orders

This code uses `asyncio` and `nest_asyncio` to run multiple asynchronous operations (`do_order` function calls for each order in `order_list`) in parallel within a single event loop, effectively managing concurrent execution.

`nest_asyncio.apply()` makes it possible to overcome limitations when running `asyncio` in environments like Jupyter notebooks, which already run in an event loop.

This method is used to efficiently process multiple orders simultaneously, reducing overall completion time compared to sequential execution.

***NOTE: This code block will sometimes throw errors, even though orderes have been submitted successfully. Be patient, and watch your Files location, for the incoming images. You should ultimately end up with the same number of images, as groups, that you caluclated earlier.***

In [ ]:
# asyncio, the Python package that provides the API to run and manage coroutines

nest_asyncio.apply()

#now all you need to do to have them run in parallel is to create an array of order requests
async with Session() as sess:
    tasks = [do_order(o) for o in order_list]
    await asyncio.gather(*tasks)

## Visualize the output!

This code block searches for "composite.tif" files in subdirectories, sorts them, and then plots them on a grid of 4x4 subplots using `matplotlib`.

Each image is opened with `rasterio`, read into an array, and displayed using imshow with a "Greens"  (Dark to Light Green) colormap. You can find more about matplotlib colormaps, [here](https://matplotlib.org/stable/users/explain/colors/colormaps.html).

The title of each subplot is set to a date extracted from the filename. Axes are turned off for clarity, and the layout is adjusted for tight spacing. This is a way to visually compare satellite images or processed data tiles based on their acquisition dates.

In [ ]:
files = []
# for folder in folder_list:
#     files.extend(glob.glob(folder+"/*/composite.tif"))

files.extend(glob.glob("*/*/composite.tif"))
files.sort()
nrow = 4
ncol = 4

f, axes = plt.subplots(nrow, ncol, figsize=(3*ncol, 3*nrow))
for file, ax in zip(files, axes.flatten()):
    with rasterio.open(file) as src:
        arr = src.read()

    ax.imshow(arr[0], cmap="Greens")


    date = file.split("_")[-1].split('/')[0]
    ax.set_title(date)

for ax in axes.flatten():
    ax.axis("off")
plt.tight_layout()

# Useful utilities

In [ ]:
# This line packages the contents of your Files folder, for download

!zip -r /content/file.zip /content/

In [ ]:
#  This code downloads the packaged files and prompts for a directory to save to.

from google.colab import files
files.download("/content/file.zip")

In [ ]:
# This function deletes all contents of a folder, recursively. USE WITH CAUTiON!!

import os
import shutil

def delete_contents(folder):
    for item in os.listdir(folder):
        item_path = os.path.join(folder, item)
        if os.path.isfile(item_path):
            os.remove(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)
    print("All files and folders have been deleted.")

# Uncomment the following line to run this funtion the /content/ folder
# in your colab notebook, or alter the directory path to target another folder

# delete_contents('/content')
